

1.   arrendondar as casas decimais certo (init)
2.   funções com valores não definidos $tg(\pi /2)$
3.   Definir multiplicação e divisão



In [2]:
!pip install git+https://github.com/brenopelegrin/LabIFSC.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/brenopelegrin/LabIFSC.git to /tmp/pip-req-build-yp1qx7o2
  Running command git clone --filter=blob:none --quiet https://github.com/brenopelegrin/LabIFSC.git /tmp/pip-req-build-yp1qx7o2
  Resolved https://github.com/brenopelegrin/LabIFSC.git to commit 595de75b033806265ae6f2b0e4f386148f73ad7f
  Preparing metadata (setup.py) ... done
  Created wheel for LabIFSC: filename=LabIFSC-0.1.13-py3-none-any.whl size=21281 sha256=4bdf29c65d20458d7ac2b4f9098854fd396da6cfd352d4bdfd6e1d18811562ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-oskeivk_/wheels/6d/ea/fa/e7a1855acf40a6fe305152eea67e6bc7e6bd763c2202f19dfd
Successfully built LabIFSC


In [109]:
from LabIFSC import*
import pkg_resources
installed= {pkg.key for pkg in pkg_resources.working_set}

def skewness(valores,desviopadrao,media):#usado para medir o quanto uma distribuição é assimétrica (ou quanto ela foge de uma gaussiana)
  m3=0
  for j in valores:
    m3+=(j-media)**3
  m3=m3/len(valores)
  g1=m3/(desviopadrao**(1.5))
  return g1

def montecarlo(funcao=sum,N=1e4,parametros=[Medida((2,0.14),"m"),Medida((4,0.1),"m")],hist=False,bins=100,assimetria="false",comparar=False):
  N=int(N)
  samples=[]
  if "numpy" in installed:
    from numpy.random import normal
    for j in range(len(parametros)): 
      samples.append(normal(parametros[j].nominal,parametros[j].incerteza,N))
  else:
    from random import gauss
    for j in range(len(parametros)): 
      samples.append(gauss(parametros[j].nominal,parametros[j].incerteza,N))
  values=[]
  for k in range(N):
    temp=[]
    for j in range(len(parametros)):
      temp+=[samples[j][k]]
    values.append(funcao(tuple(temp)))
  if "matplotlib" in installed and hist==True:
    from matplotlib.pyplot import hist
    hist(values,bins)
  if "numpy" in installed:
    from numpy import average,std
    media=average(values)
    desviopadrao=std(values)
  else:
    media=sum(values)/len(values)
    desviopadrao=0
    for j in values:
      desviopadrao+=(media-j)**2
    desviopadrao=(desviopadrao/len(values))**(1/2)
  skew=skewness(values,desviopadrao,media)
  if assimetria ==True:
    print(f"O terceiro momento estátistico é {skew}")
  metodolinear=funcao(parametros)
  if comparar==True: print(f"Esse é o resultado lienar {metodolinear}")
  return Medida((media,desviopadrao),metodolinear.unidade())
def soma(x): return x[0]/x[1]
a=Medida((10,2),"m/s^2")
b=Medida((20,4),"m")
c=montecarlo(funcao=soma,parametros=[a,b],N=1e5,comparar=True)
print(c)
        

Esse é o resultado lienar (0.5±0.2) s⁻²
(0.5±0.2) s⁻²
